#Persiapan pustaka

In [ ]:
# Import library
import pandas as pd
pd.options.mode.chained_assignment = None
pd.set_option('max_colwidth',1)
import numpy as np
seed = 0
np.random.seed(seed)
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

#Pembacaan dataset

In [ ]:
data = pd.read_csv('UlasanInDriver_4bulan.csv')
print(data.head())

                               reviewId                  userName  score  \
0  34b787d1-e79f-4512-932d-8fe78555e77b  Sasa Sasa                 3       
1  5f37918c-4ddd-4ded-996c-3f0e916804ef  Oktavian 372              5       
2  6d0af8f6-7cdb-458c-a24a-25d05fd68c5c  Tri Fandi Rezeki Damanik  3       
3  77a3c9d4-b246-45f7-a834-decf1be040b9  Ade Gilang Ramadi         5       
4  95974cdd-93fb-4f7a-a94d-28c0b30ca75b  Suwandi Wandi             5       

                    at  \
0  2024-09-30 23:46:32   
1  2024-09-30 23:46:23   
2  2024-09-30 23:29:47   
3  2024-09-30 21:01:03   
4  2024-09-30 20:38:56   

                                                                                                                  content  
0  bagus si, tp sayang d tempatku DRIVER KLO ABIS AMBIL PESENAN GA LANGSUNG OTW, ADA AJA, PDAHAL CUSTOMER NYA BURU BURU_-  
1  👍👍👍                                                                                                                     
2  Apa apa ser

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8465 entries, 0 to 8464
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   reviewId  8465 non-null   object
 1   userName  8465 non-null   object
 2   score     8465 non-null   int64 
 3   at        8465 non-null   object
 4   content   8465 non-null   object
dtypes: int64(1), object(4)
memory usage: 330.8+ KB


In [ ]:
# Hapus duplikasi berdasarkan teks asli (content)
indrive_df = data.drop_duplicates(subset=['content'])
print("Jumlah data setelah hapus duplikat awal:", indrive_df.shape[0])  # Harus 6704

Jumlah data setelah hapus duplikat awal: 6704


#Case Folding

In [ ]:
# Casefolding
import re

def text_prep(teks):
    teks = str(teks).lower()
    teks = re.sub('@[\w]+','',str(teks))  # Menghapus username (mention)
    teks = re.sub('[^a-zA-Z]','  ', str(teks))  # Menghapus karakter selain huruf
    teks = re.sub(r"\b[a-zA-Z]\b", "", teks)  # Menghapus huruf tunggal
    teks = re.sub('\s+',' ',teks)  # Menghapus spasi ekstra
    return teks

# Terapkan fungsi casefolding ke kolom 'content'
indrive_df['text_casefold'] = indrive_df['content'].apply(text_prep)

# Tampilkan hasil dalam bentuk tabel yang rapi di Google Colab
indrive_df[['content', 'text_casefold']].head(10)

,content,text_casefold
0,"bagus si, tp sayang d tempatku DRIVER KLO ABIS AMBIL PESENAN GA LANGSUNG OTW, ADA AJA, PDAHAL CUSTOMER NYA BURU BURU_-",bagus si tp sayang tempatku driver klo abis ambil pesenan ga langsung otw ada aja pdahal customer nya buru buru
1,👍👍👍,
2,"Apa apa serba mahal, tarif malah ditawar murah. Hadeh",apa apa serba mahal tarif malah ditawar murah hadeh
3,Salam nyenderr,salam nyenderr
4,Kenapa ya pencari singnal nya gak mutar diem terus,kenapa ya pencari singnal nya gak mutar diem terus
5,mantap,mantap
6,Very good,very good
7,"bagus, drivernya bersih dan harum",bagus drivernya bersih dan harum
8,Knp tempat tujuan kok tdk bisa di buka selalu saja tdk ada hasil😔,knp tempat tujuan kok tdk bisa di buka selalu saja tdk ada hasil
9,Bagus bisa kejar waktu,bagus bisa kejar waktu


#Tokenisasi

In [ ]:
# Tokenizing
import re
def tokens(teks):
    text_tokens = re.split('\W+', teks)
    return text_tokens

# Terapkan fungsi tokenizing ke kolom 'text_casefold'
indrive_df['text_token'] = indrive_df['text_casefold'].apply(lambda x: tokens(x))

# Tampilkan hasil dalam bentuk tabel yang rapi di Google Colab
indrive_df[['text_casefold', 'text_token']].head(10)

,text_casefold,text_token
0,bagus si tp sayang tempatku driver klo abis ambil pesenan ga langsung otw ada aja pdahal customer nya buru buru,"[bagus, si, tp, sayang, tempatku, driver, klo, abis, ambil, pesenan, ga, langsung, otw, ada, aja, pdahal, customer, nya, buru, buru, ]"
1,,"[, ]"
2,apa apa serba mahal tarif malah ditawar murah hadeh,"[apa, apa, serba, mahal, tarif, malah, ditawar, murah, hadeh]"
3,salam nyenderr,"[salam, nyenderr]"
4,kenapa ya pencari singnal nya gak mutar diem terus,"[kenapa, ya, pencari, singnal, nya, gak, mutar, diem, terus]"
5,mantap,[mantap]
6,very good,"[very, good]"
7,bagus drivernya bersih dan harum,"[bagus, drivernya, bersih, dan, harum]"
8,knp tempat tujuan kok tdk bisa di buka selalu saja tdk ada hasil,"[knp, tempat, tujuan, kok, tdk, bisa, di, buka, selalu, saja, tdk, ada, hasil, ]"
9,bagus bisa kejar waktu,"[bagus, bisa, kejar, waktu]"


#Normalisasi

In [ ]:
# Normalize
normalizad_word = pd.read_excel("normalisasi.xlsx")
normalizad_word_dict = {}
for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1]
def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

# Menambahkan kolom hasil normalisasi
indrive_df['text_normal'] = indrive_df['text_token'].apply(normalized_term)

# Tampilkan hasil normalisasi dalam bentuk tabel yang mudah dibaca di Google Colab
indrive_df[['text_token', 'text_normal']].head(10)

<ipython-input-13-bb4933815355>:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] not in normalizad_word_dict:
<ipython-input-13-bb4933815355>:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  normalizad_word_dict[row[0]] = row[1]


,text_token,text_normal
0,"[bagus, si, tp, sayang, tempatku, driver, klo, abis, ambil, pesenan, ga, langsung, otw, ada, aja, pdahal, customer, nya, buru, buru, ]","[bagus, si, tetapi, sayang, tempatku, driver, kalau, abis, ambil, pesenan, tidak, langsung, otw, ada, saja, pdahal, customer, nya, buru, buru, ]"
1,"[, ]","[, ]"
2,"[apa, apa, serba, mahal, tarif, malah, ditawar, murah, hadeh]","[apa, apa, serba, mahal, tarif, malah, ditawar, murah, hadeh]"
3,"[salam, nyenderr]","[salam, nyenderr]"
4,"[kenapa, ya, pencari, singnal, nya, gak, mutar, diem, terus]","[kenapa, ya, pencari, singnal, nya, tidak, mutar, diem, terus]"
5,[mantap],[mantap]
6,"[very, good]","[very, good]"
7,"[bagus, drivernya, bersih, dan, harum]","[bagus, drivernya, bersih, dan, harum]"
8,"[knp, tempat, tujuan, kok, tdk, bisa, di, buka, selalu, saja, tdk, ada, hasil, ]","[kenapa, tempat, tujuan, kok, tidak, bisa, di, buka, selalu, saja, tidak, ada, hasil, ]"
9,"[bagus, bisa, kejar, waktu]","[bagus, bisa, kejar, waktu]"


#Stopword Removal

In [ ]:
# Melakukan stopword removal atau menghilangkan kata-kata dalam sebuah kalimay yang tidak memiliki makna
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, ArrayDictionary, StopWordRemover
import pandas as pd
from IPython.display import display

# Membuat daftar stopword
stop_factory = StopWordRemoverFactory()
stopwords = stop_factory.get_stop_words()
more_stopwords = ['kenapa', 'kapan', 'bagaimana', 'apa','kok','yang','nya']
# Menggabungkan stopword default dengan stopword tambahan
data = stopwords + more_stopwords
# Membuat dictionary untuk stopword
dictionary = ArrayDictionary(data)
stopword_remover = StopWordRemover(dictionary)  # Pastikan ini diimpor dengan benar
# Daftar kata penting yang tidak boleh dihapus
important_words = set([
    "tujuan", "hasil", "waktu", "bagus", "kerja", "keputusan", "informasi",
    "penyelesaian", "proses", "bisa", "sukses", "positif", "negatif", "tepat",
    "jelas", "baik", "efektif", "cepat", "sesuai", "ideal", "produktif",
    "aplikasi", "pengguna", "layanan", "feedback", "masalah", "pengalaman", "pengemudi", "tidak",
    "penting", "kesalahan", "berhasil", "gagal", "terbaik", "terburuk",
    "lama", "baru", "terlambat", "segera", "waktu", "hari", "minggu", "bulan",
    "memuaskan", "berkualitas", "senang", "bahagia", "kesal", "marah", "frustrasi", "kecewa",
    "sedih", "puas", "tenang", "cemas", "manfaat", "fungsi", "peran", "solusi", "berusaha",
    "menyelesaikan", "menghindari", "memilih", "pilihan", "pertimbangan", "rekomendasi"
])
def stopword_removal(document):
    # Combine default stopwords with additional stopwords
    combined_stopwords = set(stopwords + more_stopwords)
    return [word for word in document if word not in combined_stopwords or word in important_words]

# Terapkan fungsi stopword removal ke kolom 'teks_norm'
indrive_df['text_stop'] = indrive_df['text_normal'].apply(stopword_removal)

# Tampilkan hasil dalam bentuk tabel yang rapi di Google Colab
display(indrive_df[['text_normal', 'text_stop']].head(10))

,text_normal,text_stop
0,"[bagus, si, tetapi, sayang, tempatku, driver, kalau, abis, ambil, pesenan, tidak, langsung, otw, ada, saja, pdahal, customer, nya, buru, buru, ]","[bagus, si, sayang, tempatku, driver, kalau, abis, ambil, pesenan, tidak, langsung, otw, pdahal, customer, buru, buru, ]"
1,"[, ]","[, ]"
2,"[apa, apa, serba, mahal, tarif, malah, ditawar, murah, hadeh]","[serba, mahal, tarif, malah, ditawar, murah, hadeh]"
3,"[salam, nyenderr]","[salam, nyenderr]"
4,"[kenapa, ya, pencari, singnal, nya, tidak, mutar, diem, terus]","[pencari, singnal, tidak, mutar, diem, terus]"
5,[mantap],[mantap]
6,"[very, good]","[very, good]"
7,"[bagus, drivernya, bersih, dan, harum]","[bagus, drivernya, bersih, harum]"
8,"[kenapa, tempat, tujuan, kok, tidak, bisa, di, buka, selalu, saja, tidak, ada, hasil, ]","[tempat, tujuan, tidak, bisa, buka, selalu, tidak, hasil, ]"
9,"[bagus, bisa, kejar, waktu]","[bagus, bisa, kejar, waktu]"


#Stemming

In [ ]:
# Import library untuk stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import pandas as pd
from IPython.display import display

# Membuat stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()
# Daftar kata yang tidak ingin diubah
no_change_words = set([
    "pengemudi", "mobil", "aplikasi", "layanan", "feedback", "masalah", "pengalaman", "semoga",
])
# Fungsi untuk stemming
def text_stem(document):
    processed_words = []
    for word in document:
        # Ubah kata menjadi huruf kecil untuk perbandingan yang konsisten
        if word.lower() in no_change_words:
            processed_words.append(word)  # Tidak mengubah kata yang ada dalam daftar
        else:
            stemmed_word = stemmer.stem(word)  # Lakukan stemming
            processed_words.append(stemmed_word)
    return processed_words

# Misalkan indrive_df adalah DataFrame yang sudah ada dan memiliki kolom 'text_stop'
# Terapkan fungsi stemming ke kolom hasil stopword removal
indrive_df['text_stem'] = indrive_df['text_stop'].apply(text_stem)

# Tampilkan hasil dalam bentuk tabel yang rapi di Google Colab
display(indrive_df[['text_stop', 'text_stem']].head(10))

,text_stop,text_stem
0,"[bagus, si, sayang, tempatku, driver, kalau, abis, ambil, pesenan, tidak, langsung, otw, pdahal, customer, buru, buru, ]","[bagus, si, sayang, tempat, driver, kalau, abis, ambil, sen, tidak, langsung, otw, pdahal, customer, buru, buru, ]"
1,"[, ]","[, ]"
2,"[serba, mahal, tarif, malah, ditawar, murah, hadeh]","[serba, mahal, tarif, malah, tawar, murah, hadeh]"
3,"[salam, nyenderr]","[salam, nyenderr]"
4,"[pencari, singnal, tidak, mutar, diem, terus]","[cari, singnal, tidak, mutar, diem, terus]"
5,[mantap],[mantap]
6,"[very, good]","[very, good]"
7,"[bagus, drivernya, bersih, harum]","[bagus, drivernya, bersih, harum]"
8,"[tempat, tujuan, tidak, bisa, buka, selalu, tidak, hasil, ]","[tempat, tuju, tidak, bisa, buka, selalu, tidak, hasil, ]"
9,"[bagus, bisa, kejar, waktu]","[bagus, bisa, kejar, waktu]"


#Simpan hasil

In [ ]:
# Menyimpan DataFrame ke file Excel dengan memilih kolom yang diinginkan
indrive_df[['content', 'text_casefold', 'text_token', 'text_normal', 'text_stop', 'text_stem']].to_excel('processed_data_final.xlsx', index=False)


In [ ]:
# Ubah kolom 'text_prep' menjadi string untuk menghindari error
indrive_df['text_clean'] = indrive_df['text_stem'].apply(lambda x: ' '.join(x))  # Jika 'text_stem' berupa list, gunakan ini

# Pilih kolom yang diperlukan
indrive_df = indrive_df[['content', 'text_clean']]  # Pastikan kolom 'content' dan 'text_clean' ada di DataFrame

# Cek kolom yang ada di DataFrame
print(indrive_df.columns)
print("Shape of DataFrame:", indrive_df.shape)

Index(['content', 'text_clean'], dtype='object')
Shape of DataFrame: (6704, 2)


In [ ]:
indrive_df[['content', 'text_clean']].to_csv('./indrive-preprocess_final.csv', encoding='utf8', index=True)